In [3]:
import pandas as pd

In [4]:
sms_spam = pd.read_excel('/Users/Isaac/Downloads/spam.xlsx')

In [5]:
sms_spam.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
print(sms_spam.shape)


(5572, 5)


In [7]:
sms_spam = sms_spam[['v1', 'v2']]

In [8]:
sms_spam.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
sms_spam.columns = ['Label', 'SMS']

In [10]:
sms_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

In [12]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1, random_state=1)

In [13]:
# Calculate index for split
training_test_index = round(len(data_randomized.index) * (4/5))


In [14]:
training_set = data_randomized[:training_test_index].reset_index(drop=True)


In [15]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1, random_state=1)
# Calculate index for split
training_test_index = round(len(data_randomized.index) * (4/5))
# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)
print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [16]:
training_set['Label'].value_counts(normalize=True)


ham     0.864065
spam    0.135935
Name: Label, dtype: float64

In [17]:
test_set['Label'].value_counts(normalize=True)


ham     0.873429
spam    0.126571
Name: Label, dtype: float64

In [18]:
training_set.head(3)

,Label,SMS
0,ham,Convey my regards to him
1,ham,"[â€_] anyway, many good evenings to u! s"
2,ham,My sort code is and acc no is . The bank is n...


In [19]:
# After cleaning
training_set['SMS'] = training_set['SMS'].str.replace(
   '\W', ' ') # Removes punctuation
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head(3)

,Label,SMS
0,ham,convey my regards to him
1,ham,â _ anyway many good evenings to u s
2,ham,my sort code is and acc no is the bank is n...


In [20]:
training_set['SMS'].head()

0                             convey my regards to him
1              â _  anyway  many good evenings to u  s
2    my sort code is  and acc no is   the bank is n...
3                          sorry i din lock my keypad 
4    hi babe its chloe  how r u  i was smashed on s...
Name: SMS, dtype: object

In [21]:
list_of_words = training_set['SMS'].astype(str).values.tolist()

In [22]:
second_list_of_words = []
for x in list_of_words:
    substrings = x.split()
    for substring in substrings:
        second_list_of_words.append(substring)

In [23]:
vocabulary = list(set(second_list_of_words))

In [24]:
len(vocabulary)

7758

In [25]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    sms = (str(sms)).split()
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [26]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,4th,parts,heads,decimal,smash,library,rgds,started,078,08712317606,...,grazed,ringtone,anymore,biro,2stop,09111030116,guai,tex,probs,comb
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,4th,parts,heads,decimal,smash,library,rgds,started,...,grazed,ringtone,anymore,biro,2stop,09111030116,guai,tex,probs,comb
0,ham,convey my regards to him,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,â _ anyway many good evenings to u s,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,my sort code is and acc no is the bank is n...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,sorry i din lock my keypad,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,spam,hi babe its chloe how r u i was smashed on s...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
training_set_clean = training_set_clean.assign(SMS=training_set.SMS.str.split(","))

In [29]:
training_set_clean.head()

,Label,SMS,4th,parts,heads,decimal,smash,library,rgds,started,...,grazed,ringtone,anymore,biro,2stop,09111030116,guai,tex,probs,comb
0,ham,[convey my regards to him],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,[ â _ anyway many good evenings to u s],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,[my sort code is and acc no is the bank is ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[sorry i din lock my keypad ],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,spam,[hi babe its chloe how r u i was smashed on ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

In [31]:
spam_messages.head()

spam_messages_copy = spam_messages.copy()

spam_messages_copy = spam_messages_copy.drop(['Label', 'SMS'], axis=1)
spam_messages_copy.head()

,4th,parts,heads,decimal,smash,library,rgds,started,078,08712317606,...,grazed,ringtone,anymore,biro,2stop,09111030116,guai,tex,probs,comb
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# N_Spam
n_words_per_spam_message = spam_messages_copy.sum(axis=1)
n_spam = n_words_per_spam_message.sum()

In [33]:
ham_messages_copy = ham_messages.copy()

ham_messages_copy = ham_messages_copy.drop(['Label', 'SMS'], axis=1)
ham_messages_copy.head()

# N_Ham
n_words_per_ham_message = ham_messages_copy.sum(axis=1)
n_ham = n_words_per_ham_message.sum()

In [34]:
n_words_per_ham_message.head()

0     5
1     9
2    25
3     6
5    15
dtype: int64

In [35]:
# N_Vocabulary
n_vocabulary = len(vocabulary)

In [36]:
# Laplace smoothing
alpha = 1

In [37]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

In [38]:
# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam

    n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham




In [39]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}
# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam

    n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

In [40]:
import re

def classify(message):
    '''
    message: a string
    '''

    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham: 
            p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [41]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 7.560202407115216e-26
P(Ham|message): 3.1285497108685822e-27
Label: Spam


In [42]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 4.919152107678309e-25
P(Ham|message): 4.049626114769741e-21
Label: Ham


In [43]:
#Measuring the Spam Filter's Accuracy
def classify_test_set(message):
    '''
    message: a string
    '''

    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return('ham')
    elif p_spam_given_message > p_ham_given_message:
        return('spam')
    else:
        return('needs human classification')

In [44]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,S...from the training manual it show there is ...,ham
1,spam,Do you want a new Video phone? 600 anytime any...,spam
2,ham,True. Its easier with her here.,ham
3,ham,Midnight at the earliest,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [45]:
#Testing Accuracy
correct = 0
total = len(test_set.index)

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1107
Incorrect: 7
Accuracy: 0.9937163375224417
